# Explanation

The recurrent neural network enables deep learning to learn about connections through time, using the backpropagation-through-time (BPTT) approach to update weights.

This paper proposes an approach to building RNNs that doesn't suffer from an infinitely growing sequence of memories that was a challenge for previous designs.

### Intuition

The primary difference between an RNN and a standard feed-forward network is just that in an RNN, inputs to neurons in one time-step can depend on both inputs to the neural network in the same time-step (common with feed-forward networks), as well as the activations of neurons from previous time-steps (this is what adds recurrence).

Thinking about the recurrent connections of the network can get confusing, especially while thinking about backpropagation, if you think about the neuron outputs looping back on themselves into the inputs.

An easier way to think about neurons in an RNN is in terms of "unrolling" the network across time, treating neurons at different time-steps as different neurons (as if they were from a separate input layer).

Thus, neurons from past time steps behave the same as neurons in previous layers in terms of back-propagation - gradients are back-propagated through previous time steps from the current time step until they reach $t = 0$.

### Math

We can define the activation of a (non-input) neuron $k$ at time $t$ as:

$$y_k(t) = f_k(s_k(t - 1)) \\ s_k(t) = \sum_{i_{U \cup I}} w_{ki}z_i(t) \\ z_k(t) =
\begin{cases}
  x_k(t) & \textrm{if } k \in I \\
  y_k(t) & \textrm{if } k \in U
\end{cases}$$

These are sufficient to explain the structure of the RNN - let's break down what they mean.

Note that each of these values are taking in $t$ rather than $x$ to show that they indicate the activations/outputs of respective units at time $t$. All input values are passed into the network and represented by the activations $x_k(t)$.

The distinction between $k \in I$ and $k \in U$ is just to show the distinction between neurons in the input layer, and neurons in the output layer (this simple construction has no hidden layer).

So $z_k(t)$ just specifies the activations for all neurons in the network by distinguishing the input from output neurons - the input $x_k(t)$ neurons depend only on their input at time step $t$, whereas the output neurons $y_k(t)$ depend on both the current time-step (via activations from $x_(t)$) as well as the previous time-step.

$s_k(t)$ trivially shows that the output neurons in $y_k(t)$ take the weighted sum of all neurons in our network, including both input neurons and output neurons (who's values are coming from the previous time-step).

$y_k(t)$ establishes the relationship that the activation of the output neurons in one time-step depends on the weighted sum of activations from the previous time-step.

Through these 3 equations, we can see the recursive link of every neuron to all previous neurons back through all time-steps until we hit $t = 0$.

Then importantly, we calculate backpropagation just as with standard feed-forward networks, except due to this recursive connection, the gradient back-propagates backward through multiple time-steps.

For weights connecting output neurons $k \in U$ to input neurons $k \in I$, nothing changes.

For weights connecting output neurons to other output neurons, gradients backpropagate through time - and we can see the final derivation of the gradient calculation on a weight:

$$\Delta w_{ij}(t) = \alpha \sum_{k \in U} e_k(t) p_{ij}^k(t) \\ p_{ij}^k(t+1) = f'_k(s_k(t))[\sum_{l \in U} w_{kl} p_{ij}^l(t) + \delta_{ik} z_j(t)] \\ p_{ij}^k(0) = 0$$

So here we see the gradients spread out fractally to increasing large trees of related weights until we reach time $t = 0$.

This is the general concept behind how RNNs work.

Notably, given the recurrence in the gradient calculation, the same weights may be passed through many times in the calculation of one gradient, accentuating large weights and causing exploding gradients as gradients move backward in time.

This problem made RNNs challenging to use in practice for a long time, until the introduction of the LSTM.



# My Notes

## 📜 [A Learning Algorithm for Continually Running Fully Recurrent Neural Networks](https://gwern.net/doc/ai/nn/rnn/1989-williams-2.pdf)

> A major problem in connectionist theory is to develop learning algorithms that can tap the full computational power of neural networks.

How do we use the full potential of neural networks? Designing architectures that enables this for different problems is challenging.

> Attention has recently turned to developing algorithms for networks with recurrent connections, which have important capabilities not found in feedforward networks, including attractor dynamics and the ability to store information for later use.

> The approach we propose here enjoys the generality of the backpropagation-through-time approach while not suffering from its growing memory requirement in arbitrarily long training sequences

### The Learning Algorithm and Variations

**1. The Basic Algorithm**

We define a network with $n$ output units, with $m$ input lines. We create the $(m + n)$-tuple $z(t)$, the concatenation of the set of input signals at time $t$, $x(t)$ and the set of output signals at time $t$, $y(t)$. Let the indices $k \in I$ represent the input units and $k \in U$ represent the output units.

$$
z_k(t) =
\begin{cases}
  x_k(t) & \textrm{if } k \in I \\
  y_k(t) & \textrm{if } k \in U
\end{cases}
$$

Let the activation of the $k$th unit at time $t$ for $k \in U$ be defined by

$$
y_k(t + 1) = f_k(s_k(t))
$$

With $f_k$ representing the neurons activation function and $s_k$ representing the sum of the neurons weights as defined by

$$
s_k(t) = \sum_{i \in U \cup I} w_{ki} z_i(t)
$$

And the notation $y_k(t+1)$ denoting that the neurons output at time-step $t$ will also be it’s contributing input to the network at time step $t + 1$.

Given $T(t)$ as the subset of neurons $k \in U$with a specific target output value $d_k(t)$ for neuron $k$ at time $t$. We can then define the error function:

$$
e_k(t) =
\begin{cases}
  d_k(t) - y_k(t) & \textrm{if } k \in T(t) \\
  0 & \textrm{otherwise}
\end{cases}
$$

> Note that this formulation allows for the possibility that target values are specified for different units at different times. The set of units considered to be visible can thus be time-varying,

Now the error function to compute the total error of the network at time $t$

$$
J(t) = 1/2 \sum_{k \in U}[e_k(t)]^2
$$

And the overall weight change for a weight $w_{ij}$ in the network

$$
\Delta w_{ij}(t) = -\alpha\frac{\partial J(t)}{\partial w_{ij}}
$$

$$
\frac{\partial{J(t)}}{\partial{w_{ij}}} = \sum_{k \in U} e_k(t) \frac{\partial{y_k(t)}}{\partial{w_{ij}}}
$$

$$
\frac{\partial{y_k(t+1)}}{\partial{w_{ij}}} = f'_k(s_k(t))[\sum_{i \in U} w_{ki} \frac{\partial{y_i(t)}}{\partial{w_{ij}}} + \delta_{ik}z_j(t)]
$$

These all obtained just through simple back-propagation on the original setup of our network using the chain-rule. The last $\frac{\partial{y_k(t+1)}}{\partial{w_{ij}}}$ representing the partial of the output neurons of the _current_ time-step with respect to the weights and outputs of the _previous_ time-step, thus enabling back-propagation recursively through time.

Then for convenience we define $p_{ij}^k$ where $k \in U$, $i \in U$, $j \in U \cup I$, $k$ denoting the output neuron who’s gradient to back-propagate to the _previous_ time-step, and $i$, $j$ specifying the weight between an input and output neuron in the previous time-step.

$$
p_{ij}^k(t+1) = f'_k(s_k(t))[\sum_{l \in U} w_{kl} p_{ij}^l(t) + \delta_{ik} z_j(t)]
$$

Thus, the gradient recursively back-propagates through previous time-steps via the recursive use of $p_{ij}^l$ in it’s own definition, hitting a base case at $t = 0$ where we define the the value to be 0 since the output neuron values at this point are un-tethered to any previous time-step.

$$
p_{ij}^k(t_0) = 0
$$

Then, we have the final weight update algorithm at a given time-step $t$

$$
\Delta w_{ij}(t) = \alpha \sum_{k \in U} e_k(t) p_{ij}^k (t)
$$

**2. Real-Time Recurrent Learning**

> In order to allow real-time training of behaviors of indefinite duration, however, it is useful to relax this assumption and actually make the weight changes while the network is running.

In practice, we update weights at every time-step, rather than accumulating gradients and then changing them at epochs.

> While the resulting algorithm is no longer guaranteed to follow the gradient of total error, the practical differences are often slight, with the two versions becoming more nearly identical as the learning rate is made smaller.

**3. Teacher-Forced Real-Time Recurrent Learning**

> An interesting technique that is frequently used in dynamical supervised learning tasks is to replace the actual output $y_k(t)$ of a unit by the teacher signal $d_k(t)$ in subsequent computation of the behavior of the network, whenever such a value exists. We call this technique _teacher forcing_.

### Discussion

> Our primary goal here has been to derive a learning algorithm to train completely recurrent, continually updated networks to learn temporal tasks.

> Our emphasis has been on using uniform starting configurations that contain no a priori information about the temporal nature of the task.

> The solutions found by the algorithm are often dauntingly obscure, particularly for complex tasks involving internal state. This observation is already familiar in work with feedforward networks. This obscurity has often limited our ability to analyze the solutions in sufficient detail.

Interpretability has been near impossible, even from the beginning.
